### Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math 

### Importing Data (list of static S&P500 stocks)

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [ ]:
print(stocks)

### Obtaining API

In [ ]:
from secrets1 import AV_PRICE_API_TOKEN
from secrets2 import AV_MKTCAP_API_TOKEN

### Calling one API

In [ ]:
symbol='AAPL'
api_price_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={AV_PRICE_API_TOKEN}'
data_price = requests.get(api_price_url).json()
data_price 

In [ ]:
api_mktcap_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={AV_MKTCAP_API_TOKEN}'
data_mktcap = requests.get(api_mktcap_url).json()
data_mktcap

In [ ]:
data_price['Global Quote']['05. price']
data_mktcap['MarketCapitalization']

In [ ]:
print(data_price['Global Quote']['05. price'])
print(data_mktcap['MarketCapitalization'])

### Looping through Tickers in S&P500 Stocks

In [ ]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
df = pd.DataFrame(columns = my_columns)

In [ ]:
for symbol in stocks['Ticker']:
    api_price_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={AV_PRICE_API_TOKEN}'
    api_mktcap_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={AV_MKTCAP_API_TOKEN}'
    data_price = requests.get(api_price_url).json()
    data_mktcap = requests.get(api_mktcap_url).json()
    new_row = pd.DataFrame([[symbol, data_price['Global Quote']['05. price'], data_mktcap['MarketCapitalization'], 'N/A']],
                       columns=my_columns)
    df = pd.concat([df, new_row], ignore_index=True)

df

### Calculating number of shares to buy

In [ ]:
while True:
    portfolio_size = input("Please enter the value of your portfolio: ")
    
    try:
        val = float(portfolio_size)
        break  # Exit the loop if the input is a valid float
    except ValueError:
        print("That's not a number! Please try again.")

In [ ]:
position_size = float(portfolio_size) / len(df.index)
for i in range(0, len(df.index):
    df.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / df['Price'][i])
df

### Formatting Excel Output

In [ ]:
writer = pd.ExcelWriter('Equal_Weights_S&P500.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()